In [54]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from torch.utils.data import Dataset, DataLoader
from scipy.sparse import csr_matrix

print("--- Environment Check ---")
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())
print("Current CUDA device:", torch.cuda.current_device() if torch.cuda.is_available() else "N/A")
print()


--- Environment Check ---
PyTorch version: 2.5.1+cu124
CUDA available: True
CUDA device count: 1
Current CUDA device: 0



In [55]:
df = pd.read_csv("../Training Dataset/final_dataset.csv")
X = df["Sentence"].values
y = df["Label"].values

valid_idx = [i for i, txt in enumerate(X) if isinstance(txt, str)]
X = X[valid_idx]
y = y[valid_idx]

print("--- XSS Detection Model Demonstration ---")
print(f"Loaded {len(X)} samples successfully")
print(f"Number of positive samples: {sum(y)}")
print(f"Number of negative samples: {len(y) - sum(y)}")

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.33, random_state=42)

print("Using device:", "cuda" if torch.cuda.is_available() else "cpu")
print()
print("Dataset splits:")
print(f"Training: {len(X_train)} samples")
print(f"Validation: {len(X_val)} samples")
print(f"Test: {len(X_test)} samples")
print()


--- XSS Detection Model Demonstration ---
Loaded 88309 samples successfully
Number of positive samples: 50589
Number of negative samples: 37720
Using device: cuda

Dataset splits:
Training: 61816 samples
Validation: 17750 samples
Test: 8743 samples



In [56]:
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_val_vec   = vectorizer.transform(X_val)
X_test_vec  = vectorizer.transform(X_test)


In [57]:
class SparseDataset(Dataset):
    def __init__(self, X_sparse, y):
        self.X = X_sparse
        self.y = y

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        x_dense = self.X[idx].toarray().squeeze()
        return torch.tensor(x_dense, dtype=torch.float32), torch.tensor(self.y[idx], dtype=torch.long)

class MLPModel(nn.Module):
    def __init__(self, input_dim):
        super(MLPModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 2)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)
        return x


In [58]:
train_dataset = SparseDataset(X_train_vec, y_train)
val_dataset   = SparseDataset(X_val_vec,   y_val)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=512, shuffle=False)

input_dim = X_train_vec.shape[1]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

lrs = [0.001, 0.002, 0.01, 0.02, 0.05]
EPOCHS = 50

all_train_losses = {}
all_val_losses   = {}

for lr in lrs:
    model = MLPModel(input_dim).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    train_losses = []
    val_losses   = []

    print(f"--- Learning Rate: {lr} ---")
    for epoch in range(1, EPOCHS+1):
        model.train()
        total_loss = 0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            out = model(xb)
            loss = criterion(out, yb)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_train_loss = total_loss / len(train_loader)

        model.eval()
        val_loss_total = 0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                out = model(xb)
                vloss = criterion(out, yb)
                val_loss_total += vloss.item()
        avg_val_loss = val_loss_total / len(val_loader)

        train_losses.append(avg_train_loss)
        val_losses.append(avg_val_loss)
        print(f"Epoch {epoch}/{EPOCHS}: Train Loss = {avg_train_loss:.4f}, Val Loss = {avg_val_loss:.4f}")

    all_train_losses[lr] = train_losses
    all_val_losses[lr]   = val_losses

    # 單組 loss 圖
    plt.figure(figsize=(12,6))
    plt.plot(range(1, EPOCHS+1), train_losses, label="Train Loss")
    plt.plot(range(1, EPOCHS+1), val_losses,   label="Validation Loss")
    plt.title(f"MLP Loss (lr={lr})")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)
    plt.savefig(f"MLP_loss_plot_lr_{lr}.png")
    plt.close()

    # 存模型
    torch.save(model.state_dict(), f"MLP_model_lr_{lr}.pth")


--- Learning Rate: 0.001 ---
Epoch 1/50: Train Loss = 0.1513, Val Loss = 0.0285
Epoch 2/50: Train Loss = 0.0176, Val Loss = 0.0339
Epoch 3/50: Train Loss = 0.0085, Val Loss = 0.0421
Epoch 4/50: Train Loss = 0.0055, Val Loss = 0.0518
Epoch 5/50: Train Loss = 0.0044, Val Loss = 0.0713
Epoch 6/50: Train Loss = 0.0040, Val Loss = 0.0728
Epoch 7/50: Train Loss = 0.0033, Val Loss = 0.0851
Epoch 8/50: Train Loss = 0.0031, Val Loss = 0.0842
Epoch 9/50: Train Loss = 0.0030, Val Loss = 0.0758
Epoch 10/50: Train Loss = 0.0026, Val Loss = 0.0980
Epoch 11/50: Train Loss = 0.0023, Val Loss = 0.0988
Epoch 12/50: Train Loss = 0.0022, Val Loss = 0.1031
Epoch 13/50: Train Loss = 0.0022, Val Loss = 0.1115
Epoch 14/50: Train Loss = 0.0023, Val Loss = 0.1105
Epoch 15/50: Train Loss = 0.0022, Val Loss = 0.1138
Epoch 16/50: Train Loss = 0.0020, Val Loss = 0.1221
Epoch 17/50: Train Loss = 0.0019, Val Loss = 0.1265
Epoch 18/50: Train Loss = 0.0018, Val Loss = 0.1265
Epoch 19/50: Train Loss = 0.0019, Val Loss =

In [59]:
plt.figure(figsize=(12,6))
for lr in lrs:
    plt.plot(range(1, EPOCHS+1), all_train_losses[lr], label=f"Train lr={lr}")
plt.title("MLP Combined Training Losses")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.savefig("MLP_combined_training_losses.png")
plt.close()

plt.figure(figsize=(12,6))
for lr in lrs:
    plt.plot(range(1, EPOCHS+1), all_val_losses[lr], label=f"Val lr={lr}")
plt.title("MLP Combined Validation Losses")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.savefig("MLP_combined_validation_losses.png")
plt.close()
print("✅ All done.")


✅ All done.
